In [ ]:
print("Hello World!")

In [21]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = "datasets/housing"
HOUSING_URL = DOWNLOAD_ROOT+HOUSING_PATH+"/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL,housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url,tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
import pandas as pd

def load_housing_data(housing_path = HOUSING_PATH):
    csv_path = os.path.join(housing_path,"housing.csv")
    return pd.read_csv(csv_path)

In [22]:
#fetch_housing_data()
housing = load_housing_data()
#housing.head()

In [ ]:
#housing.info()

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# housing.hist(bins=50,figsize=(20,15))
# plt.show()

In [ ]:
# import numpy as np
# def split_train_test(data,test_ratio):
#     shuffled_indices = np.random.permutation(len(data))
#     test_set_size = int(len(data)*test_ratio)
#     test_indices = shuffled_indices[:test_set_size]
#     train_indices = shuffled_indices[test_set_size:]
#     return data.iloc[train_indices],data.iloc[test_indices]
# train_set,test_set = split_train_test(housing,0.2)
# print(len(train_set),"train+",len(test_set),"test")

In [ ]:
# import hashlib
# def test_set_check(identifier,test_ratio,hash):
#     return hash(np.int64(identifier)).digest()[-1]<256*test_ratio
# def split_train_test_by_id(data,test_ratio,id_cloumn,hash=hashlib.md5):
#     ids =data[id_cloumn]
#     in_test_set = ids.apply(lambda id_:test_set_check(id_,test_ratio,hash))
#     return data.loc[~in_test_set],data.loc[in_test_set]
# housing_with_id = housing.reset_index()
# #train_set,test_set = split_train_test_by_id(housing_with_id,0.2,"index")
# housing_with_id["id"] = housing["longitude"]*1000+housing["latitude"]
# train_set,test_set = split_train_test_by_id(housing_with_id,0.2,"id")
# print(len(train_set),"train+",len(test_set),"test")

In [ ]:
# from sklearn.model_selection import train_test_split

# train_set,test_set = train_test_split(housing,test_size=0.2,random_state=42)
# print(len(train_set),"train+",len(test_set),"test")

In [ ]:
import numpy as np
housing["income_cat"]=np.ceil(housing["median_income"]/1.5)
housing["income_cat"].where(housing["income_cat"]<5,5.0,inplace=True)
# import pandas as pd
# housing["income_cat"] = pd.cut(housing["median_income"],bins=[0., 1.5, 3.0, 4.5, 6., np.inf],labels=[1, 2, 3, 4, 5])

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# housing["income_cat"].value_counts()/len(housing)

In [ ]:
# for set in(strat_train_set,strat_test_set): #删除income_cat,数据恢复原样
#     set.drop(["income_cat"],axis=1,inplace=True) # axis=1 表示列

In [ ]:
# 上边完成了测试集的探索，下边开始训练集
# housing = strat_train_set.copy()

In [ ]:
#housing.plot(kind="scatter",x="longitude",y="latitude",alpha=0.1)

In [ ]:
# housing.plot(kind="scatter",x="longitude",y="latitude",alpha=0.4,s=housing["population"]/100,label="population",
#             c="median_house_value",cmap=plt.get_cmap("jet"),colorbar=True)
# plt.legend()

In [ ]:
#corr_matrix=housing.corr()
#corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
#from pandas.tools.plotting import scatter_matrix

#attributes = ["median_house_value","median_income","total_rooms","housing_median_age"]
#scatter_matrix(housing[attributes],figsize=(12,8))

In [ ]:
#housing.plot(kind="scatter",x="median_income",y="median_house_value",alpha=0.1)

In [ ]:
# housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
# housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
# housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
#corr_matrix = housing.corr()
#corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# housing = strat_train_set.drop("median_house_value", axis=1)
# housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
# housing.dropna(subset=["total_bedrooms"]) # option 1
# housing.drop("total_bedrooms", axis=1) # option 2
# median = housing["total_bedrooms"].median()
# housing["total_bedrooms"].fillna(median) # option 3

In [23]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
housing_num = housing.drop("ocean_proximity", axis=1)
# imputer.fit(housing_num)
# imputer.statistics_
# housing_num.median().values

In [ ]:
# X = imputer.transform(housing_num)
# housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# housing_cat = housing["ocean_proximity"]
# housing_cat_encoded = encoder.fit_transform(housing_cat)
# housing_cat_encoded

In [ ]:
# print(encoder.classes_)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
# housing_cat_1hot

In [ ]:
# housing_cat_1hot.toarray()

In [ ]:
# from sklearn.preprocessing import LabelBinarizer
# encoder = LabelBinarizer(sparse_output=True) # 参数表示稀疏矩阵
# housing_cat_1hot = encoder.fit_transform(housing_cat)
# housing_cat_1hot

In [26]:
from sklearn.base import BaseEstimator,TransformerMixin

rooms_ix,bedrooms_ix,population_ix,household_ix = 3,4,5,6

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
    def __init__(self,add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self,X,y=None):
        return self # nothing else to do
    def transform(self,X,y=None):
        rooms_per_household = X[:,rooms_ix]/ X[:,household_ix]
        population_per_household = X[:,population_ix] / X[:,household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:,bedrooms_ix] / X[:,rooms_ix]
            return np.c_[X,rooms_per_household,population_per_household,bedrooms_per_room]
        else:
            return np.c_[X,rooms_per_household,population_per_household]
        
# attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
# housing_extra_attribs = attr_adder.transform(housing.values)

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline=Pipeline([('imputer',SimpleImputer(strategy="median")),('attribs_adder',CombinedAttributesAdder()),('std_scaler',StandardScaler())])
# housing_num_tr = num_pipeline.fit_transform(housing_num)

In [28]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()
housing_num = housing.drop("ocean_proximity", axis=1)

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([("num",num_pipeline,num_attribs),("cat",OneHotEncoder(),cat_attribs),])
housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
# housing_prepared

In [ ]:
# housing_prepared.shape

In [ ]:
# from sklearn.base import BaseEstimator, TransformerMixin

# # Create a class to select numerical or categorical columns 
# class OldDataFrameSelector(BaseEstimator, TransformerMixin):
#     def __init__(self, attribute_names):
#         self.attribute_names = attribute_names
#     def fit(self, X, y=None):
#         return self
#     def transform(self, X):
#         return X[self.attribute_names].values

In [ ]:
# from sklearn.base import TransformerMixin #gives fit_transform method for free
# class MyLabelBinarizer(TransformerMixin):
#     def __init__(self, *args, **kwargs):
#         self.encoder = LabelBinarizer(*args, **kwargs)
#     def fit(self, X, y=None):
#         self.encoder.fit(X)
#         return self
#     def transform(self, X, y=None):
#         return self.encoder.transform(X)

In [ ]:
# from sklearn.pipeline import FeatureUnion

# num_attribs = list(housing_num)
# cat_attribs = ["ocean_proximity"]

# old_num_pipeline = Pipeline([('selector',OldDataFrameSelector(num_attribs)),('imputer',SimpleImputer(strategy="median")),('attribs_adder',CombinedAttributesAdder()),('std_scaler',StandardScaler()),])
# #old_cat_pipeline = Pipeline([('selector',OldDataFrameSelector(cat_attribs)),('label_binarizer',MyLabelBinarizer()),])
# old_cat_pipeline = Pipeline([('selector', OldDataFrameSelector(cat_attribs)),('cat_encoder', OneHotEncoder(sparse=False)),])
# old_full_pipeline = FeatureUnion(transformer_list=[("num_pipeline",old_num_pipeline),("cat_pipeline",old_cat_pipeline),])

# old_housing_prepared = old_full_pipeline.fit_transform(housing)
# old_housing_prepared

In [ ]:
# np.allclose(housing_prepared, old_housing_prepared)#比较两个array是不是每一元素都相等，默认在1e-05的误差范围内

# Select and Train a Model

In [31]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [32]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions: ",lin_reg.predict(some_data_prepared))

Predictions:  [203682.37379543 326371.39370781 204218.64588245  58685.4770482
 194213.06443039]


In [33]:
print("Labels: ",list(some_labels))

Labels:  [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [34]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels,housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68376.64295459937

In [38]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [39]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

In [41]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [44]:
def display_scores(scores):
    print("Scores:",scores)
    print("Mean:",scores.mean())
    print("Standard deviation:",scores.std())
display_scores(tree_rmse_scores)

Scores: [69989.23152758 67321.7560422  71341.91652499 69350.10589526
 69874.58900874 76956.00739894 70619.29720446 71409.39406289
 77248.5470047  69059.58620234]
Mean: 71317.04308720981
Standard deviation: 3102.38107463059


In [45]:
lin_scores = cross_val_score(lin_reg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [66877.52325028 66608.120256   70575.91118868 74179.94799352
 67683.32205678 71103.16843468 64782.65896552 67711.29940352
 71080.40484136 67687.6384546 ]
Mean: 68828.9994844933
Standard deviation: 2662.761570610343


In [47]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared,housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels,housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18720.465107987588

In [48]:
forest_scores = cross_val_score(forest_reg,housing_prepared,housing_labels,scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [49604.15641063 47884.14063109 49714.66215194 52144.64942176
 49548.81670784 53051.30871295 49035.95732986 48103.31251254
 53211.71554049 50065.53009386]
Mean: 50236.42495129483
Standard deviation: 1819.7743842600858


In [ ]:
import joblib
joblib.dump(my_model,"my_model.pkl")
my_model_loaded = joblib.load("my_model.pkl")